# Space X Dash

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-1-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

In [3]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},                                          
                                                ],
                                                value='ALL',
                                                placeholder="place holder here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(0, 10000, 1000, id='payload-slider',value=['min_payload','max_payload']),
                                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                                    names='Launch Site', 
                                    title='Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.pie(filtered_df, values=[filtered_df['class'].mean(),1-filtered_df['class'].mean()],
                                    names=['Success','Failed'],
                                    title=entered_site)
        return fig
        # return the outcomes piechart for a selected site

In [6]:
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])

        
def get_scatter_plot(entered_site, entered_payload):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        filtered_df = filtered_df[(filtered_df['Payload Mass (kg)']<entered_payload[1])&(filtered_df['Payload Mass (kg)']>entered_payload[0])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)',y='class',
                                    #names='scatter chart names', 
                                    title='Correlation between Payload and Success for all Sites',
                                    color="Booster Version Category")
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df[(filtered_df['Payload Mass (kg)']<entered_payload[1])&(filtered_df['Payload Mass (kg)']>entered_payload[0])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)',y='class', 
                                    #names='scatter chart names', 
                                    title=entered_site,
                                    color="Booster Version Category")
        return fig

In [7]:
       
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2022 20:45:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_1_0m1645185896.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/deps/react@16.v2_1_0m1645185896.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_1_0m1645185896.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_1_0m1645185896.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_1_0m1645185896.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_1_0m1645185896.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:16] "GET /_dash-component-sui

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_funct

127.0.0.1 - - [23/Feb/2022 20:45:16] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [23/Feb/2022 20:45:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:21] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/robertosanzgarcia/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_funct

127.0.0.1 - - [23/Feb/2022 20:45:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [23/Feb/2022 20:45:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 20:45:46] "POST /_dash-update-component HTTP/1.1" 200 -
